In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import optimizers
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

2023-11-01 10:46:06.634402: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-01 10:46:08.237216: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 10:46:08.237512: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 10:46:08.246307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-01 10:46:09.016218: I tensorflow/core/platform/cpu_feature_g

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)
x_train=np.expand_dims(x_train,axis=-1)
x_test=np.expand_dims(x_test,axis=-1)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)


In [5]:


def squeeze_excite_block2D(filters,input):         
    print(input.shape)
    se = tf.keras.layers.GlobalAveragePooling2D()(input)
    print(se.shape)
    se = tf.keras.layers.Reshape((1, filters))(se) 
    print(se.shape)
    se = tf.keras.layers.Dense(filters//32, activation='relu')(se)
    print(se.shape)
    se = tf.keras.layers.Dense(filters, activation='sigmoid')(se)
    print(se.shape)
    se = tf.keras.layers.multiply([input, se])
    print(se.shape)
    return se



In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, shear_range=10,
                        height_shift_range=0.1, zoom_range=0.2)
datagen.fit(x_train)
datagen2 = tf.keras.preprocessing.image.ImageDataGenerator()
datagen2.fit(x_test)

In [7]:
def make_model():
        s = tf.keras.Input(shape=x_train.shape[1:]) 
        print(s.shape)
        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(s)
        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        print(x.shape)
        x = tf.keras.layers.BatchNormalization()(x)
        print(x.shape)
        x = squeeze_excite_block2D(128,x)
        print(x.shape)

        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = squeeze_excite_block2D(128,x)
        x = tf.keras.layers.AveragePooling2D(2)(x)


        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = squeeze_excite_block2D(128,x)
        x = tf.keras.layers.AveragePooling2D(2)(x)


        x = tf.keras.layers.concatenate([tf.keras.layers.GlobalMaxPooling2D()(x),
                                         tf.keras.layers.GlobalAveragePooling2D()(x)])

        x = tf.keras.layers.Dense(10,activation='softmax',use_bias=False,
                                  kernel_regularizer=tf.keras.regularizers.l1(0.00025))(x) # this make stacking better
        return tf.keras.Model(inputs=s, outputs=x)

In [10]:
batch_size=32
supermodel=[]
for i in range(20):
        np.random.seed(i)
        model=make_model()                
        model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
        model.fit(datagen.flow(x_train, y_train, batch_size=batch_size,shuffle=True),
                    steps_per_epoch=len(x_train) / batch_size, epochs=13,verbose=0)
        model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
        model.fit(datagen.flow(x_train, y_train, batch_size=batch_size,shuffle=True),
                    steps_per_epoch=len(x_train) / batch_size, epochs=3,verbose=0)
        model.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy',metrics=['accuracy'])
        model.fit(datagen.flow(x_train, y_train, batch_size=batch_size,shuffle=True),
                    steps_per_epoch=len(x_train) / batch_size, epochs=3,verbose=0)
        model.fit(x_train, y_train, batch_size=batch_size,shuffle=True, epochs=1,verbose=0)
        supermodel.append(model)
        print(i,'acc:',accuracy_score(np.argmax(y_test,axis=1),np.argmax(model.predict(x_test),axis=1)))

(None, 28, 28, 1)
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 128)
(None, 1, 128)
(None, 1, 4)
(None, 1, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 128)
(None, 1, 128)
(None, 1, 4)
(None, 1, 128)
(None, 28, 28, 128)
(None, 14, 14, 128)
(None, 128)
(None, 1, 128)
(None, 1, 4)
(None, 1, 128)
(None, 14, 14, 128)
